In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
import joblib
import json

df = pd.read_csv("../data/processed/marketing_campaign_cleaned.csv")
df.shape, df.columns

((2240, 35),
 Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
        'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
        'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
        'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
        'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
        'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
        'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response',
        'Customer_Duration', 'Total_Kids', 'Total_Spent', 'TotalAcceptedCmp',
        'Campaign_Engaged', 'Avg_Spend_Per_Visit'],
       dtype='object'))

In [3]:
df_encoded = pd.get_dummies(df, drop_first=True)
df_encoded.head()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,Dt_Customer_2014-06-20,Dt_Customer_2014-06-21,Dt_Customer_2014-06-22,Dt_Customer_2014-06-23,Dt_Customer_2014-06-24,Dt_Customer_2014-06-25,Dt_Customer_2014-06-26,Dt_Customer_2014-06-27,Dt_Customer_2014-06-28,Dt_Customer_2014-06-29
0,5524,1957,58138.0,0,0,58,635,88,546,172,...,False,False,False,False,False,False,False,False,False,False
1,2174,1954,46344.0,1,1,38,11,1,6,2,...,False,False,False,False,False,False,False,False,False,False
2,4141,1965,71613.0,0,0,26,426,49,127,111,...,False,False,False,False,False,False,False,False,False,False
3,6182,1984,26646.0,1,0,26,11,4,20,10,...,False,False,False,False,False,False,False,False,False,False
4,5324,1981,58293.0,1,0,94,173,43,118,46,...,False,False,False,False,False,False,False,False,False,False


In [4]:
X = df_encoded.drop("Response", axis=1)
y = df_encoded["Response"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [5]:
#compute scale_pos_weight
pos = y_train.sum()
neg = len(y_train) - pos
scale = neg / pos

xgb_model = XGBClassifier(scale_pos_weight=scale, use_label_encoder=False, eval_metric="logloss", random_state=42)
xgb_model.fit(X_train, y_train)

/Volumes/Projects/Portfolio/marketing-analytics-dashboard/venv/lib/python3.11/site-packages/xgboost/training.py:183: UserWarning: [22:34:23] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [6]:
y_pred = xgb_model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[361  20]
 [ 35  32]]
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       381
           1       0.62      0.48      0.54        67

    accuracy                           0.88       448
   macro avg       0.76      0.71      0.73       448
weighted avg       0.87      0.88      0.87       448



In [8]:
joblib.dump(xgb_model, "../models/xgboost_response_model.pkl")
with open("../models/feature_columns.json", "w") as f:
    json.dump(list(X_train.columns), f)

print("Model and feature list saved")

Model and feature list saved.
